<a href="https://colab.research.google.com/github/KeisukeShimokawa/CarND-Advanced-Lane-Lines/blob/master/part1/lesson12_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf

In [6]:
tf.__version__

'1.15.0'

## Hello World

In [0]:
# tensorflow用のオブジェクトを生成する
hello_constant = tf.constant('hello world')

In [10]:
with tf.Session() as sess:
    # セッション内で`tf.constant`の操作を実行する
    output = sess.run(hello_constant)
    print(output)

b'hello world'


## Tensor

tensorflowには整数型や文字列などのデータ型は存在しておらず、すべて`tensor`というオブジェクトで操作できる。

先ほどの`hello_constant`は0次元の文字列のtensorである。

In [0]:
# 0次元の32ビット整数
A = tf.constant(1234)

# 1次元の32ビット整数
B = tf.constant([123, 456, 789])

# 2次元の32ビット整数
C = tf.constant([[123, 456, 789], [222, 333, 444]])

## Session

tensorflowの考え方のもとは**計算グラフ**です。
tensorflowのコードをじっこうすると、計算の流れのイメージは以下のようになります。

![session](https://i.gyazo.com/03a2ca22fa6f0509612bfcf1a14df22a.png)

つまりこの`Session`の中で計算グラフを構築し、セッション内でGPUやCPUの操作などが可能になります。

In [12]:
with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

b'hello world'


## Input

では定数ではなく変数を使用するには、`tf.placeholder`と`feed_dict`を使用して、セッション内でデータをtensorflowに与えていきます。

tensorflowには、異なるデータセットを異なるパラメータで持つことができるようにデータセットとして`x`などをそのまま設定することはできません。

そこで`tf.placeholder`を使用して、`tf.session.run()`が実行された際に渡されるデータを格納する形で、tensorオブジェクトを事前に用意しておく必要があります。

`tf.session.run()`の中で`feed_dict`パラメータを使用すれば、事前に用意しているtensorオブジェクトに実際のデータを格納できます。

In [13]:
x = tf.placeholder(tf.string)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello World'})
    print(output)

Hello World


In [16]:
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Test String', y: 123, z: 45.67})
    print(output)

Test String


## Math

[公式](https://www.tensorflow.org/api_docs/python/tf/math)

四則演算もtensorflowで実行可能です。

In [18]:
# 整数やtensorを受け取って加算演算を行う。
x = tf.add(5, 2)

with tf.Session() as sess:
    output = sess.run(x)
    print(output)

7


In [19]:
# 整数やtensorを受け取って減算演算を行う。
x = tf.subtract(10, 4)

with tf.Session() as sess:
    output = sess.run(x)
    print(output)

6


In [20]:
# 整数やtensorを受け取って乗算演算を行う。
x = tf.multiply(5, 2)

with tf.Session() as sess:
    output = sess.run(x)
    print(output)

10


In [22]:
# 以下はtensorの型が異なるのでエラーが生じる
tf.subtract(tf.constant(2.0), tf.constant(1))

TypeError: ignored

In [24]:
# tf.cast(original, dtype)で型を合わせればエラーは生じない
tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))

<tf.Tensor 'Sub_4:0' shape=() dtype=int32>

In [25]:
# Quiz Solution
# Note: You can't run code in this tab
import tensorflow as tf

# TODO: Convert the following to TensorFlow:
x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.divide(x,y),tf.cast(tf.constant(1), tf.float64))

# TODO: Print z from a session
with tf.Session() as sess:
    output = sess.run(z)
    print(output)

4.0


## Classification

ロジスティックモデルでは、入力に対して重みをかけてバイアス項を足したものが各ラベルに対する活性値になります。

![](https://i.gyazo.com/5a69ca4d4efd56c7e6c1f3752a8b4f57.png)

しかし単に活性値なだけであると解釈が難しいので、ソフトマックス関数を設けることで確率としてとらえることができるようになります。

![](https://i.gyazo.com/10612ac8e909617a07380c2557300ffa.png)

計算上は`y=Wx+b`と表示されることが多いですが、tensorflowは内部的に`y=xW+b`を採用しているため、こちらで計算を進めていきます。行列がベクトルを線形変換する機能だと考えれば、自然と行列の計上もわかります。

### tf.Variable()

`tf.placeholder`や`tf.constant`は後から修正することができないため、パラメータを使用する場合には`tf.Variable()`を使用します。

また`tf.global_variables_initializer()`を使用することで、パラメータの初期値を決めることが可能です。初期化には、最初は単純に正規分布にのっとって計算する方法があります。

正規分布から乱数をサンプリングする際には`tf.truncated_normal()`を使用します。

バイアス項などの初期値を0で設定しても構わない変数に対しては、`tf.zeros()`を使用します。



In [0]:
# Quiz Solution
import tensorflow as tf

def get_weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # TODO: Return weights
    return tf.Variable(tf.truncated_normal((n_features, n_labels)))


def get_biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # TODO: Return biases
    return tf.Variable(tf.zeros(n_labels))


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # TODO: Linear Function (xW + b)
    return tf.add(tf.matmul(input, w), b)

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

def mnist_features_labels(n_labels):

    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

    # バッチデータに説明変数と目的変数が格納されているのでそれぞれを抽出する
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # any()によってn_labelsが5の場合に[0, 0, 0, 0, 1, 0, 0, ...]のようなときに
        # [0, 0, 0, 0, 1]のように5番目までを抽出できる。
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels

In [44]:
# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels
n_labels = 3

# Features and Labels
features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

# Weights and Biases
w = get_weights(n_features, n_labels)
b = get_biases(n_labels)

# Linear Function xW + b
logits = linear(features, w, b)

# Training data
train_features, train_labels = mnist_features_labels(n_labels)

Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz


In [45]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    # Softmax
    prediction = tf.nn.softmax(logits)

    # Cross entropy
    # This quantifies how far off the predictions were.
    # You'll learn more about this in future lessons.
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

    # Training loss
    # You'll learn more about this in future lessons.
    loss = tf.reduce_mean(cross_entropy)

    # Rate at which the weights are changed
    # You'll learn more about this in future lessons.
    learning_rate = 0.08

    # Gradient Descent
    # This is the method used to train the model
    # You'll learn more about this in future lessons.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Run optimizer and get loss
    _, l = session.run(
        [optimizer, loss],
        feed_dict={features: train_features, labels: train_labels})

# Print loss
print('Loss: {}'.format(l))

Loss: 6.842278003692627
